In [2]:
import functools
import numpy as np
import sklearn.datasets
import sklearn.model_selection
from sklearn.metrics import mean_squared_error

import pso
import ann


In [36]:
def dim_weights(shape):
    dim = 0
    for i in range(len(shape)-1):
        dim = dim + (shape[i] + 1) * shape[i+1]
    return dim


def weights_to_vector(weights):
    w = np.asarray([])
    for i in range(len(weights)):
        v = weights[i].flatten()
        w = np.append(w, v)
    return w


def vector_to_weights(vector, shape):
    weights = []
    idx = 0
    for i in range(len(shape)-1):
        r = shape[i+1]
        c = shape[i] + 1
        idx_min = idx
        idx_max = idx + r*c
        W = vector[idx_min:idx_max].reshape(r,c)
        weights.append(W)
    return weights


#this is the optimisation for the coords of the particles??
def eval_neural_network(weights, shape, X, y, activationLayer):
    mse = np.asarray([])
    for w in weights:
        weights = vector_to_weights(w, shape)
        nn = ann.MultiLayerPerceptron(shape, weights=weights)
        y_pred = nn.run(X,activationLayer)
        mse = np.append(mse, mean_squared_error(np.atleast_2d(y), y_pred))
    return mse


def print_best_particle(best_particle):
    print("New best particle found at iteration #{i} with mean squared error: {score}".format(i=best_particle[0], score=best_particle[1]))


def to_one_of_k(y):
    num_inputs = 2
    y_true = np.zeros((len(y), num_classes))

    for i in range(len(y)):
        y_true[i, y[i]] = 1

    y_test_true = np.zeros((len(y_test), num_classes))
    for i in range(len(y_test)):
        y_test_true[i, y_test[i]] = 1
    
    return y_true


def twospirals(n_points, noise=.0, square=False, sine=False):
    """
     Returns the two spirals dataset.
    """
    n_points = int(n_points/2)
    n = np.sqrt( np.random.rand(n_points,1) ) * 780 * (2*np.pi)/360
    d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise
    d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise
    d1 = (d1x, d1y)
    negd1 = (-d1x, -d1y)

    if sine: 
        sinx1, sinx2 = np.sin(d1x), np.sin(d1y)
        d1 += (sinx1, sinx2)
        negd1 += (-sinx1, -sinx2)
    if square:
        sqx1, sqx2 = d1x*d1x, d1y*d1y
        d1 += (sqx1, sqx2)
        negd1 += (-sqx1, -sqx2)

    x1 = np.hstack(d1)
    x2 = np.hstack(negd1)
    x = np.vstack( (x1, x2) )
    y = np.hstack( ( np.zeros(n_points), np.ones(n_points) ) )
    return (x, y.astype(int))
    # (np.vstack( ( np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)) ) ), np.hstack((np.zeros(n_points),np.ones(n_points))))

#just xy error = 0.22938741634873416
#with sinx sin y error = 0.2325498216710004
#with sin and sq error = 0.21790465807135315

In [38]:


X_train, y_train = twospirals(n_points=100)
X_test, y_test = twospirals(n_points=100)

y_true_2d =  to_one_of_k(y=y_test) # to_one_of_k


# X = X_train
# y = y_train

In [39]:
num_classes = len(np.unique(y_train))
num_inputs = X_train.shape[1]



In [ ]:
array = [0.51,0.49,0.6]
print(np.round(array,0))

In [ ]:
def training(shape, activationLayer):

    cost_func = functools.partial(eval_neural_network, shape=shape, X=X, y=y_true.T,activationLayer=activationLayer)

    swarm = pso.ParticleSwarm(cost_func, num_dimensions=dim_weights(shape), num_particles=25,chi=0.72984 ,phi_p=2.05,phi_g=2.05)
    # Train...
    i = 0
    best_scores = [(i, swarm.best_score)]
    print_best_particle(best_scores[-1])
    
    trainingLoss = []
    testloss = []
    iterations = 2000
    while swarm.best_score>1e-6 and i<iterations:
        swarm._update()
        i = i+1
        best_weights = vector_to_weights(swarm.g, shape)
        best_nn = ann.MultiLayerPerceptron(shape, weights=best_weights)

        y_test_pred =np.round(best_nn.run(X_test,activationLayer),0)
        testError = sklearn.metrics.mean_squared_error(y_test_true,y_test_pred.T)
        
        #print("index = ",i)
        if i%500==0 and swarm.best_score < best_scores[-1][1]:
            best_scores.append((i, swarm.best_score))
            print_best_particle(best_scores[-1])
            #training per epoch
            print("Mean absolute error score for index   =",i,(activationLayer,testError))
        trainingLoss.append(swarm.best_score)
        testloss.append(testError)
        
    # Test...
    
    best_weights = vector_to_weights(swarm.g, shape)
    best_nn = ann.MultiLayerPerceptron(shape, weights=best_weights)

    y_test_pred =np.round(best_nn.run(X_test,activationLayer),0)
    
    points = np.arange(iterations)
    plt.plot(points,testloss,color="b",label="Test loss")
    plt.plot(points,trainingLoss,color="r",label="Train loss")
    plt.xlabel("Iterations")
    plt.ylabel("Error")
    plt.title(activationLayer)
    plt.legend()
    plt.show()
    ##this is dumb
    #eval neural network is pritning the mean squared error at each time
    print("best score = ",best_scores[ -1])
  #  print(sklearn.metrics.classification_report(y_test_true, y_test_pred.T))

    print("Mean absolute error score for  =",(activationLayer, sklearn.metrics.mean_squared_error(y_test_true,y_test_pred.T)))
    #print("Accuracy score = ",sklearn.metrics.accuracy_score(y_test_true,y_test_pred.T))
    return sklearn.metrics.mean_squared_error(y_test_true,y_test_pred.T)


In [ ]:
shape=(num_inputs,6,num_classes)
activationLayer="relu"
training(shape,activationLayer)
# training(shape,"sigmoid")

In [ ]:
def testRun():
    '''
    I want mse to be y axis
    I want neurons to be x axis
    then I will have 3 plots, tanh, sigmoid and relu?
    '''
    #shape = (num_inputs, 8, num_classes)

    activations = ['sigmoid','tanh']#,'relu','default']
    neurons = [4,5,6,7,8]
    arrays = np.zeros(shape=(len(activations),len(neurons)))

    for i in range(len(arrays)):
        activationLayer = activations[i]
        for k in range(len(neurons)):
            shape = (num_inputs,neurons[k],num_classes)
            mse = training(shape,activationLayer)
            arrays[i][k]=mse
            print(i,k)
     #   print(i)

    return arrays

arrays = testRun()



In [ ]:
# def testRun():
#     '''
#     I want mse to be y axis
#     I want neurons to be x axis
#     then I will have 3 plots, tanh, sigmoid and relu?
#     '''
#     #shape = (num_inputs, 8, num_classes)

#     activations = ['tanh']#,'relu','default']
#     neurons = [4,5,6,7,8]
#     arrays = np.zeros(shape=(len(activations),len(neurons)))

#     for i in range(len(arrays)):
#         activationLayer = activations[i]
#         for k in range(len(neurons)):
#             shape = (num_inputs,neurons[k],num_classes)
#             mse = training(shape,activationLayer)
#             arrays[i][k]=mse
#             print(i,k)
#      #   print(i)

#     return arrays

# tanh = testRun()


In [ ]:
import matplotlib.pyplot as plt

neurons = [4,5,6,7,8]
plt.figure(figsize=(7,7))
plt.plot(neurons,arrays[1],color='r',label="tanh")
plt.plot(neurons,arrays[0],color='g',label="sigmoid")
#plt.plot(neurons,arrays[2],color='c',label="relu")
#plt.plot(neurons,arrays[3],color='b',label="Identity")
plt.xlabel("Neurons")
plt.ylabel("Test error: MSE")
plt.legend()
plt.show()
print("hello")

In [ ]:
'''
Using the brute force method figure out the optimal functions

then use the same functions in PSO and compare it with base line

'''

In [ ]:
import matplotlib.pyplot as plt

neurons = [4,5,6,7,8]
plt.figure(figsize=(7,7))
plt.plot(neurons,arrays[1],color='r',label="tanh")
plt.plot(neurons,arrays[0],color='g',label="sigmoid")
#plt.plot(neurons,arrays[2],color='c',label="relu")
#plt.plot(neurons,arrays[3],color='b',label="Identity")
plt.xlabel("Neurons")
plt.ylabel("Test error: MSE")
plt.legend()
plt.show()
print("hello")